<a href="https://colab.research.google.com/github/deepakawl/teaching/blob/main/SunOil_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sun Oil: Regional Facility Configuration

SunOil, a global manufacturer of petrochemicals is considering several options to meet demand. One possibility is to set up a facility in each region. This would lower transportation costs and helps to avoid tariffs and duties, but foregoes benefits of the economies of scale. The table below contains annual demand, variable production and transportation cost (including tariffs), as well as the fixed cost of establishing a facility.

How should SunOil plan its production network? Consider a possibility when an additional 50% tariff is imposed for importing goods from Asia to North America.

| Supply Region | N. America | S. America | Europe | Asia | Africa |
|---------------|------------|------------|--------|------|--------|
| N. America    | 81         | 92         | 101    | 130  | 115    |
| S. America    | 117        | 77         | 108    | 98   | 100    |
| Europe        | 102        | 105        | 95     | 119  | 111    |
| Asia          | 115        | 125        | 90     | 59   | 74     |
| Africa        | 142        | 100        | 103    | 105  | 71     |
| Demand        | 12         | 8          | 14     | 16   | 7      |

In [10]:
tariff = 0          ### Add tariff in decimal format such as 0, 0.1.

In [17]:
# Install and import packages
#!pip install tabulate
#!pip install gurobipy
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate

In [18]:
### RAWDATA
# Cost matrix
cost_mat = np.array([[ 81,  92, 101, 130, 115],
                     [117,  77, 108,  98, 100],
                     [102, 105,  95, 119, 111],
                     [115, 125,  90,  59,  74],
                     [142, 100, 103, 105,  71]
                    ])

# Adjust the cost of import from Asia using tariff
cost_mat[3,0] = cost_mat[3,0]*(1+tariff)

# Demand in each location
demand = np.array([12, 8, 14, 16, 7])

# Capacity and Fixed costs for each supply line (low and high- capacity)
lcap_fc = np.array([6000, 4500, 6500, 4100, 4000])
q_lcap = np.array([10, 10, 10, 10, 10])

hcap_fc = np.array([9000, 6750, 9750, 6150, 6000])
q_hcap = np.array([20, 20, 20, 20, 20])

In [26]:
# identify number of supply and demand location for iterations
n_supply = range(len(cost_mat))
n_demand = range(len(cost_mat[0]))

# Initial value of decision variables
dec_qty = {(i, j): 0 for i in n_supply for j in n_demand}
dec_lcap = {(i): 0 for i in n_supply}
dec_hcap = {(i): 0 for i in n_supply}

# Create a Gurobi model
model = Model("MinimizeCost")

# Define decision variables, enforce integer type with lower bound = 0
# There are three sets of decisions- 1) units to produce and transport from supply to demand location, 2) low capacity line 3) high capacity line at supply locations
dec_qty = {(i, j): model.addVar(vtype=GRB.INTEGER, lb=0, name=f"Quantity_{i}_{j}") for i in n_supply for j in n_demand}
dec_lcap = {(i): model.addVar(vtype=GRB.INTEGER, name=f"Dec_Low_{i}") for i in n_supply}
dec_hcap = {(i): model.addVar(vtype=GRB.INTEGER, name=f"Dec_High_{i}") for i in n_supply}

# Update the model
model.update()

# Set objective function - Total cost = Production & Transportation cost across network + Fixed cost of low cap line + fixed cost of high cap line
model.setObjective(
    quicksum(cost_mat[i, j] * dec_qty[i, j] for i in n_supply for j in n_demand) +
    quicksum(lcap_fc[i] * dec_lcap[i] for i in n_supply) +
    quicksum(hcap_fc[i] * dec_hcap[i] for i in n_supply),
    GRB.MINIMIZE
)

# Excess Capacity constraints
for i in n_supply:
    model.addConstr(dec_lcap[i] * q_lcap[i] + dec_hcap[i] * q_hcap[i] - (sum(dec_qty[i, j] for j in n_demand)) >= 0, f"Excess_Capacity_Constraints_{i}")

# Unmet demand constraints
for j in n_demand:
    model.addConstr(demand[j] - (sum(dec_qty[i, j] for i in n_supply)) <= 0, f"Unmet_Demand_Constraints_{j}")

# Suppress optimization output
model.Params.OutputFlag = 0

# Optimize the model
model.optimize()

# Extract results to print as table
op_qty  = [[dec_qty[i, j].x for j in n_demand] for i in n_supply]
op_lcap = [dec_lcap[i].x for i in n_supply]
op_hcap = [dec_hcap[i].x for i in n_supply]
lowhigh = [dec_lcap[i].x + dec_hcap[i].x for i in n_supply]

# Display results as a table
src_loc = ["N.America", "S.America", "Europe", "Asia", "Africa"]
headers = ["N.America"] + ["S.America"] + ["Europe"] + ["Asia"] + ["Africa"] + ["Low"] + ["High"] + ["Total"]
table_data = [[src_loc[i]] + row + [op_lcap[i], op_hcap[i], lowhigh[i]] for i, row in enumerate(op_qty)]
table = tabulate(table_data, headers=headers, tablefmt="grid")

print(table)
print(f"\n@ Tariff = {tariff*100}%, Minimum Cost: $ {model.objVal}")

+-----------+-------------+-------------+----------+--------+----------+-------+--------+---------+
|           |   N.America |   S.America |   Europe |   Asia |   Africa |   Low |   High |   Total |
+===========+=============+=============+==========+========+==========+=======+========+=========+
| N.America |          -0 |          -0 |       -0 |     -0 |       -0 |    -0 |     -0 |      -0 |
+-----------+-------------+-------------+----------+--------+----------+-------+--------+---------+
| S.America |          -0 |          -0 |       -0 |     -0 |       -0 |    -0 |     -0 |      -0 |
+-----------+-------------+-------------+----------+--------+----------+-------+--------+---------+
| Europe    |          -0 |          -0 |       -0 |     -0 |       -0 |    -0 |     -0 |      -0 |
+-----------+-------------+-------------+----------+--------+----------+-------+--------+---------+
| Asia      |          12 |           0 |       12 |     16 |        0 |    -0 |      2 |       2 |
